#  PySpark for ETL


The CSV files were downloaded into Databricks File System (DBFS), and then were converted into Parquet files via Koalas for better efficiency.

Download url: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page.

Data dictionary: https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf.

## Download CSV files to DBFS

In [0]:
%sh rm -r /dbfs/FileStore/taxi_parquet/

rm: cannot remove '/dbfs/FileStore/taxi_parquet/': No such file or directory
/bin/bash: line 1: fg: no job control
/bin/bash: line 2: fg: no job control

In [0]:
%sh mkdir -p /dbfs/FileStore/taxi_csv/

In [0]:
url_loc = {} # Map download url to the file location in DBFS

year = 2020
for m in range(1, 13):
   month = "{:02d}".format(m)
   fname = 'yellow_tripdata_%s-%s.csv' % (year, month)
   url = 'https://s3.amazonaws.com/nyc-tlc/trip+data/%s' % fname
   loc = '/dbfs/FileStore/taxi_csv/%s' % fname
   url_loc[url] = loc

In [0]:
import urllib.request

for url, loc in url_loc.items():
  urllib.request.urlretrieve(url, loc)

In [0]:
# Defining schema
from pyspark.sql.types import IntegerType , StringType , DoubleType , TimestampType ,StructType ,StructField
schema_df = StructType([ 
    StructField("VendorID",IntegerType(),True), 
    StructField("tpep_pickup_datetime",TimestampType(),True), 
    StructField("tpep_dropoff_datetime",TimestampType(),True), 
    StructField("passenger_count", IntegerType(), True), 
    StructField("trip_distance", DoubleType(), True), 
    StructField("RatecodeID", IntegerType(), True), 
    StructField("store_and_fwd_flag",StringType(),True), 
    StructField("PULocationID",IntegerType(),True), 
    StructField("DOLocationID",IntegerType(),True), 
    StructField("payment_type", IntegerType(), True), 
    StructField("fare_amount", DoubleType(), True), 
    StructField("extra", DoubleType(), True), 
    StructField("mta_tax",DoubleType(),True), 
    StructField("tip_amount",DoubleType(),True), 
    StructField("tolls_amount",DoubleType(),True), 
    StructField("improvement_surcharge", DoubleType(), True), 
    StructField("total_amount", DoubleType(), True), 
    StructField("congestion_surcharge", DoubleType(), True) 
  ])

In [0]:
# Creating dataframe from all CSV files for 2020
csv_Df  = spark.read.schema(schema_df).option("header",'true').csv(path = 'file:/dbfs/FileStore/taxi_csv')

In [0]:
csv_Df.select("tpep_dropoff_datetime").show(2)

+---------------------+
tpep_dropoff_datetime|
+---------------------+
 2020-02-01 00:30:32|
 2020-02-01 01:05:36|
+---------------------+
only showing top 2 rows

In [0]:
# Data cleaning and transformations
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import col,quarter,to_timestamp,hour
from pyspark.sql.types import LongType
# Duplicates values are dropped
csv_Df=csv_Df.drop_duplicates()
# Transformation are applied
csv_Df = csv_Df.withColumn("tip_percent",(col("tip_amount")/col("total_amount"))*100).withColumn("quarter",quarter(csv_Df.tpep_dropoff_datetime)).withColumn("trip_duration",(col("tpep_dropoff_datetime").cast("long") - col("tpep_pickup_datetime").cast("long"))/60).withColumn("trip_hour",hour(col("tpep_dropoff_datetime"))).withColumn("speed",(col("trip_distance").cast("long"))/col("trip_duration").cast("long"))

In [0]:
# Writing the final data in parquet format 
sql("SET spark.databricks.delta.formatCheck.enabled=false")
csv_Df.coalesce(1).write.parquet(path='/FileStore/yellow_taxi_parquet')

In [0]:
# Writting dataframe to SQLLITE table yellow_taxi_data
#df_parquet.write.format("jdbc").mode("overwrite").options(url ="jdbc:sqlite:file:/dbfs/FileStore/DWH_task.db", driver="org.sqlite.JDBC", dbtable="yellow_taxi_data").save()



--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3487693547477984> in <module> 
 1 #spark.read.format("jdbc").options(url ="jdbc:sqlite:file:/dbfs/FileStore/DWH_task.db", driver="org.sqlite.JDBC", dbtable="yellow_taxi_data").load().take(10) 
 2 #spark.stop() 
 ----> 3 df_parquet . write . format ( "jdbc" ) . mode ( "overwrite" ) . options ( url = "jdbc:sqlite:file:/dbfs/FileStore/DWH_task.db" , driver = "org.sqlite.JDBC" , dbtable = "yellow_taxi_data" ) . save ( ) 
 4 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 1132 self . format ( format ) 
 1133 if path is None : 
 -> 1134 self . _jwrite . save ( ) 
 1135 else : 
 1136 self . _jwrite . save ( path ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o3395.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 13.0 failed 1 times, most recent failure: Lost task 4.0 in stage 13.0 (TID 69) (ip-10-172-194-83.us-west-2.compute.internal executor driver): java.sql.SQLException: database is locked
	at org.sqlite.core.DB.throwex(DB.java:859)
	at org.sqlite.core.DB.executeBatch(DB.java:760)
	at org.sqlite.core.CorePreparedStatement.executeBatch(CorePreparedStatement.java:77)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.savePartition(JdbcUtils.scala:687)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$saveTable$1(JdbcUtils.scala:856)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$saveTable$1$adapted(JdbcUtils.scala:854)
	at org.apache.spark.rdd.RDD.$anonfun$foreachPartition$2(RDD.scala:1025)
	at org.apache.spark.rdd.RDD.$anonfun$foreachPartition$2$adapted(RDD.scala:1025)
	at org.apache.spark.SparkContext.$anonfun$runJob$2(SparkContext.scala:2486)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$1(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:150)
	at org.apache.spark.scheduler.Task.$anonfun$run$1(Task.scala:119)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.Task.run(Task.scala:91)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$13(Executor.scala:788)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1643)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:791)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:647)
	at java.util.concurrent.T